In [166]:
import sys
sys.path.append(r"D:\MINJI\NETWORK RELIABILITY\BNS-JT-python")

import networkx as nx
import matplotlib.pyplot as plt
from BNS_JT import cpm, variable, operation
from networkx.algorithms.flow import shortest_augmenting_path
from BNS_JT import brc
import numpy as np

In [167]:
# Node and edge data
nodes = {
    "n1": (0, 0),
    "n2": (10, 10),
    "n3": (10, -10),
    "n4": (20, 0)
}

edges = {  
    "e1": ("n1", "n2"),   
    "e2": ("n2", "n1"),   
    "e3": ("n1", "n3"),   
    "e4": ("n3", "n1"),   
    "e5": ("n2", "n3"),   
    "e6": ("n3", "n2"),   
    "e7": ("n2", "n4"),   
    "e8": ("n4", "n2"),   
    "e9": ("n3", "n4"),   
    "e10": ("n4", "n3")   
}

# Probability of failure/survival (failure: 0, survival: 1)
probs = {
    'e1': {0: 0.01, 1: 0.99}, 
    'e2': {0: 0.01, 1: 0.99}, 
    'e3': {0: 0.01, 1: 0.99},
    'e4': {0: 0.01, 1: 0.99},
    'e5': {0: 0.05, 1: 0.95},
    'e6': {0: 0.05, 1: 0.95}, 
    'e7': {0: 0.05, 1: 0.95}, 
    'e8': {0: 0.05, 1: 0.95},
    'e9': {0: 0.10, 1: 0.90}, 
    'e10': {0: 0.10, 1: 0.90}
}

# Initial intact capacity
intact_capacity = { 
    "e1": 15, "e2": 15, 
    "e3": 15, "e4": 15,
    "e5": 7.5, "e6": 7.5,
    "e7": 15, "e8": 15, 
    "e9": 15, "e10": 15
}

# Function to generate random component states (0 or 1) based on failure probabilities
def generate_comps_st(probs):
    comps_st = {}
    for edge, prob in probs.items():
        comps_st[edge] = np.random.choice([0, 1], p=[prob[0], prob[1]])  # Randomly choose based on probabilities
    return comps_st

# arc_capacity = comps_st * intact_capacity
comps_st = generate_comps_st(probs)
arc_capacity = {edge: intact_capacity[edge] * comps_st[edge] for edge in intact_capacity}

# Compute distances
def euclidean_distance(node1, node2):
    x1, y1 = nodes[node1]
    x2, y2 = nodes[node2]
    return round(((x2 - x1)**2 + (y2 - y1)**2)**0.5, 2)

arc_distance = {edge_name: euclidean_distance(u, v) for edge_name, (u, v) in edges.items()}   

# Create the graph
G = nx.DiGraph()

for node, position in nodes.items():
    G.add_node(node, pos=position)

for edge_name, (u, v) in edges.items(): 
    distance = arc_distance[edge_name]   
    capacity = arc_capacities[edge_name]  
    G.add_edge(u, v, weight=distance, capacity=capacity)   

# Calculate maximum allowable distance
# Demand data
demand = {
    'k1': {'origin': 'n1', 'destination': 'n4', 'amount': 28},
    'k2': {'origin': 'n2', 'destination': 'n3', 'amount': 18},
    'k3': {'origin': 'n1', 'destination': 'n3', 'amount': 11},
    'k4': {'origin': 'n3', 'destination': 'n4', 'amount': 16},
    'k5': {'origin': 'n4', 'destination': 'n1', 'amount': 20},
    'k6': {'origin': 'n3', 'destination': 'n2', 'amount': 14},
    'k7': {'origin': 'n3', 'destination': 'n1', 'amount': 11},
    'k8': {'origin': 'n4', 'destination': 'n3', 'amount': 18}
}

# Maximum allowable delay time 6 minutes / average velocity 149 km/h
avg_velo = 149
for commodity, info in demand.items():
    shortest_distance = nx.shortest_path_length(G, source=info['origin'], target=info['destination'], weight='weight')
    max_allowable_time = (shortest_distance * 60) / avg_velo + 6  # 6 minutes extra
    max_distance = max_allowable_time * avg_velo / 60

# Plot the network
plt.figure(figsize=(8, 6))
pos = nx.get_node_attributes(G, 'pos')
nx.draw_networkx_nodes(G, pos, node_size=450, node_color="lightblue")
nx.draw_networkx_edges(
    G, pos,
    edgelist=list(edges.values()),  
    arrowstyle='-|>',
    arrowsize=15,
    connectionstyle='arc3,rad=0.1',
    min_target_margin=10,
    min_source_margin=10
)
edge_labels = {(u, v): f"{arc_distance[edge_name]} km, Cap: {arc_capacities[edge_name]}" 
               for edge_name, (u, v) in edges.items()}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8, label_pos=0.5)
nx.draw_networkx_labels(G, pos, font_size=9)
plt.title("Network with Bidirectional Arcs")
plt.show()

# Print edge information
print("\nEdge Information:")
for edge_name, (u, v) in edges.items():
    edge_data = G.get_edge_data(u, v) 
    capacity = edge_data['capacity']
    print(f"{edge_name} Capacity: {capacity}")



Edge Information:
e1 Capacity: 15
e2 Capacity: 15
e3 Capacity: 15
e4 Capacity: 15
e5 Capacity: 7.5
e6 Capacity: 7.5
e7 Capacity: 15
e8 Capacity: 15
e9 Capacity: 15
e10 Capacity: 15


In [168]:
varis = {}
for k, v in edges.items():
    varis[k] = variable.Variable( name=k, values = [0, 1]) # values: edge flow capacity

In [169]:
def MCNF_systemfunc(comps_st, edges, arc_capacity, demand, max_distance, arc_distance):
    from gurobipy import Model, GRB, quicksum

    # Create Gurobi model
    model = Model("Network Flow Optimization")
      
    # Define variables
    arcs = [(edges[edge][0], edges[edge][1]) for edge in edges]  # (i, j) 형식의 arc 리스트
    flow = {}
    unmet_demand = {}

    for k, info in demand.items():
        unmet_demand[k] = model.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"unsatisfied_{k}")
        for i, j in arcs:
            capacity = arc_capacity.get((i, j), 0)
            flow[k, i, j] = model.addVar(lb=0, ub=capacity, vtype=GRB.CONTINUOUS, name=f"flow_{k}_{i}_{j}")

    # Objective function: Minimize expected loss
    cost_coefficient = {
        k: info['amount'] * nx.shortest_path_length(G, source=info['origin'], target=info['destination'], weight='weight') * 0.1723
        for k, info in demand.items()
    }
    
    model.setObjective(
        quicksum(cost_coefficient[k] * unmet_demand[k] for k in demand),
        GRB.MINIMIZE
    )

    # Extract all nodes from edge values
    nodes = set(node for edge in edges.values() for node in edge)

    # Constraint 1: Flow conservation
    for k, info in demand.items():
        origin = info['origin']
        destination = info['destination']
        amount = info['amount']
        for node in nodes: 
            inflow = quicksum(flow[k, i, j] for i, j in arcs if j == node)
            outflow = quicksum(flow[k, i, j] for i, j in arcs if i == node)
            if node == origin:
                model.addConstr(outflow - inflow == amount - unmet_demand[k])
            elif node == destination:
                model.addConstr(outflow - inflow == - amount + unmet_demand[k])
            else:
                model.addConstr(outflow - inflow == 0)

    # Constraint 2: Arc capacity limits
    for i, j in arcs:
        model.addConstr(quicksum(flow[k, i, j] for k in demand) <= arc_capacity.get((i, j), 0))

    # Constraint 3: Distance limits
    for k, info in demand.items():
        origin = info['origin']
        distance_expr = quicksum(arc_distance.get((i, j), 0) * flow[k, i, j] for i, j in arcs)
        total_flow = quicksum(flow[k, i, j] for i, j in arcs if i == origin)
        model.addConstr(distance_expr <= max_distance * total_flow)

    # Perform optimization
    model.optimize()

    # Process results
    if model.status == GRB.OPTIMAL:
        expected_loss = model.objVal
        flows = {
            k: {arc: flow[k, arc[0], arc[1]].X for arc in arcs} for k in demand
        }

        # Define system state based on expected_loss
        if expected_loss <= 2400:
            sys_st = 'threshold 1'
        elif 2400 < expected_loss <= 2600:
            sys_st = 'threshold 2'
        elif 2600 < expected_loss <= 2800:
            sys_st = 'threshold 3'
        elif 2800 < expected_loss <= 3000:
            sys_st = 'threshold 4'
        else:
            sys_st = 'threshold 5'

        return expected_loss, sys_st, flows
    else:
        return None, None, None

In [170]:
# Solve MCNF optimization for the current sample
sys_fun = lambda comps_st: MCNF_systemfunc(
    comps_st=comps_st,
    edges=edges,
    arc_capacity={  # 수정: arc_capacity를 (i, j) 형태로 저장
        (edges[edge][0], edges[edge][1]): intact_capacity[edge] * comps_st[edge]
        for edge in intact_capacity
    },
    demand=demand,
    max_distance=max_distance,
    arc_distance=arc_distance
)

expected_loss, sys_st, flows = sys_fun(comps_st)  
print(f"Expected Loss: {expected_loss}, System State: {sys_st}")
print("Flows:", flows)


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 5 3600XT 6-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 50 rows, 88 columns and 276 nonzeros
Model fingerprint: 0x706f70b1
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e+01, 1e+02]
  Bounds range     [8e+00, 2e+01]
  RHS range        [8e+00, 3e+01]
Presolve removed 27 rows and 27 columns
Presolve time: 0.00s
Presolved: 23 rows, 61 columns, 141 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8126236e+03   4.225000e+01   0.000000e+00      0s
      21    3.7398508e+03   0.000000e+00   0.000000e+00      0s

Solved in 21 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.739850758e+03
Expected Loss: 3739.850758000001, System State: threshold 5
Flows: {'k1': {('n1', 'n2'): 15.0, ('n2', 'n1'): 0.0, (

In [171]:
brs, rules, sys_res, monitor = brc.run(varis, probs, sys_fun, max_sf=np.inf, max_nb=np.inf, pf_bnd_wr=0.0)

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 5 3600XT 6-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 50 rows, 88 columns and 276 nonzeros
Model fingerprint: 0xd22da378
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [3e+01, 1e+02]
  Bounds range     [8e+00, 2e+01]
  RHS range        [8e+00, 3e+01]
Presolve removed 24 rows and 16 columns
Presolve time: 0.00s
Presolved: 26 rows, 72 columns, 170 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   3.625000e+01   0.000000e+00      0s
      23    2.1294419e+03   0.000000e+00   0.000000e+00      0s

Solved in 23 iterations and 0.01 seconds (0.00 work units)
Optimal objective  2.129441916e+03


KeyError: 'threshold 1'